In [0]:
# install the required bert pretrained model
!pip install pytorch-pretrained-bert pytorch-nlp

     |████████████████████████████████| 133kB 7.5MB/s 
     |████████████████████████████████| 92kB 9.5MB/s 


In [0]:
import re
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def preprocess_tweet(tweet):
	#Preprocess the text in a single tweet
	#arguments: tweet = a single tweet in form of string 
	#convert the tweet to lower case
	tweet.lower()
	#convert all urls to sting "URL"
	tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
	#convert all @username to "at_user"
  #tweet = re.sub('@[^\s]+','AT_USER', tweet)
	#correct all multiple white spaces to a single white space
	tweet = re.sub('[\s]+', ' ', tweet)
	#convert "#topic" to just "topic"
	tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
	return tweet

In [0]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
model.cuda()

In [0]:
model_save_name = 'classifier.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
model.load_state_dict(torch.load(path))
model.eval()

In [0]:
df4 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/testdata.manual.2009.06.14.csv',encoding='utf-8')

In [0]:
df4 = df4[['@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.','4']]
df4.rename(columns={'@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.':'text'},inplace=True)
df4.rename(columns={'4':'sentiment'},inplace=True)
#df4.head()
sentences1 = df4.text.values
def convert_to_year(date_in_some_format):
  if int(date_in_some_format) == 4:
    date_in_some_format = 1
  return int(date_in_some_format)

df4['sentiment'] = df4['sentiment'].apply(convert_to_year)
#df4.head()


In [0]:

df4['text'] = df4['text'].apply(preprocess_tweet)
#df4.head()
sentences1 = ["[CLS] " + sentence + " [SEP]" for sentence in sentences1]
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts1 = [tokenizer.tokenize(sent) for sent in sentences1]
#print ("Tokenize the first sentence:")
#print (tokenized_texts1[0])

MAX_LEN = df4.text.map(lambda x: len(x)).max() + 12

input_ids1 = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts1],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids1 = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts1]
input_ids1 = pad_sequences(input_ids1, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
attention_masks1 = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids1:
  seq_mask = [float(i>0) for i in seq]
  attention_masks1.append(seq_mask)
input_ids1
labels = df4.sentiment.values
type(labels)
type(labels[0])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)
prediction_inputs = torch.tensor(input_ids1)
prediction_masks = torch.tensor(attention_masks1)
prediction_labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 4

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
model.eval()
# Tracking variables 
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# Evaluate data for one epoch
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up validation
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
  eval_accuracy += tmp_eval_accuracy
  nb_eval_steps += 1

print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Validation Accuracy: 0.734


In [0]:
!pip install emoji
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download('averaged_perceptron_tagger')

In [0]:
import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks/')
from old_tweets1 import *
df5 = collect_tweets("samsung")

all modules uceesflly imported
emoji
                                                text              location
0  Wow I'm eligible to win a Samsung Galaxy S20 ....              Dehradun
1  Check out this Amazon deal: Galaxy Tab A 8.0 2...                      
2  @SamsungHelpUK just got on board with Samsung ...  Somewhere in England
3  @JustinStoutArt Definitely redundant. It's not...         Cleveland, OH
4  Samsung Galaxy A21 Specifications Suggested on...                      
lower
                                                text              location
0  wow i'm eligible to win a samsung galaxy s20 ....              Dehradun
1  check out this amazon deal: galaxy tab a 8.0 2...                      
2  @samsunghelpuk just got on board with samsung ...  Somewhere in England
3  @justinstoutart definitely redundant. it's not...         Cleveland, OH
4  samsung galaxy a21 specifications suggested on...                      
punct
                                                tex

In [0]:
sentences1 = df5.text.values
#df5.head(10)
#print(sentences1)
sentences1 = ["[CLS] " + sentence + " [SEP]" for sentence in sentences1]
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts1 = [tokenizer.tokenize(sent) for sent in sentences1]
#print ("Tokenize the first sentence:")
#print (tokenized_texts1[0])

MAX_LEN = df5.text.map(lambda x: len(x)).max()+10



input_ids1 = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts1],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids1 = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts1]
input_ids1 = pad_sequences(input_ids1, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
attention_masks1 = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids1:
  seq_mask = [float(i>0) for i in seq]
  attention_masks1.append(seq_mask)
input_ids1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)
prediction_inputs = torch.tensor(input_ids1)
prediction_masks = torch.tensor(attention_masks1)
batch_size = 4

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
i = -1
for batch in prediction_dataloader:
  i += 1
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up validation
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  pred_flat = np.argmax(logits, axis=1).flatten()
  def find_this(pred1):
    if pred1 == 1:
      v = "positive"
    elif pred1 == 0:
      v = "negative"
    else:
      v = "neutral"
    return v
  k = -1
  for j in range(i*batch_size,(i*batch_size)+batch_size):
    k += 1
    print(df5['text'][j],find_this(pred_flat[k]))
  #print(df5['text'][i*batch_size:(i*batch_size)+batch_size],pred_flat)
  #3break
  if i == 3:
    break



wow im eligible win samsung galaxy s20 cool quiztimemorningswithamazon samsunggalaxys20 httpstcozabijdxb3r neutral
check amazon deal galaxy tab 80 2019 rugged case kickstandsmt290 smt295 poetic full body shoc… httpstcopgx1vx0vti neutral
samsunghelpuk got board samsung cloud thought id use handy download feature windows its… httpstcofireovqgcf neutral
justinstoutart definitely redundant companies caring ecosystem apple… httpstcovendksijpf negative
samsung galaxy a21 specifications suggested benchmark site httpstcoljfyz4j2s3 httpstcowsaev59ade neutral
galaxys20quiz surprise samsung httpstcoms4vc857nf positive
1raghvan1 thank writing request use support channels team happy h… httpstcogaslsdmayz positive
samsung e apple httpstcoghajkipmlm neutral
ready stock flipcoverbookcoversilicon andromaxsamsung galaxyblackberryiphone fast respone add pin 32a834ab 08891047101 positive
hibiscus amp moon captured 03 dec 2017 full moon night day time using samsung e5… httpstcorzxy3xvi8p neutral
today cass

In [0]:
#print(df5['text'][9],pred)